In [1]:
# Installing necessary libraries
!pip install -q -U google-genai
!pip install -q rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 8.1 MB/s eta 0:00:00


In [2]:
# config & imports
import os
import time
import logging
from google import genai
from google.genai import types
from rich.console import Console
from rich.markdown import Markdown

# logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(message)s')
logger = logging.getLogger("TripVerse")

console = Console()

# client setup
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GEMINI_API_KEY = user_secrets.get_secret("GEMINI_API_KEY")
    client = genai.Client(api_key=GEMINI_API_KEY)
    console.print("api key loaded and client initialized")
except Exception as e:
    console.print("could not load api key")

api key loaded and client initialized

In [3]:
# tool one: google search
google_search_tool = types.Tool(
    google_search=types.GoogleSearch()
)

# tool two: custom currency converter
import requests

def get_rates():
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    EXCHANGE_RATE_API_KEY = user_secrets.get_secret("EXCHANGE_RATE_API_KEY")
    url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/latest/USD"
    data = requests.get(url).json()
    return data["conversion_rates"]

def currency_converter(amount, from_currency, to_currency):
    rates = get_rates()
        
    if from_currency not in rates or to_currency not in rates:
        return f"Error: Exchange rate not available for {from_currency} → {to_currency}."
    
    amount_in_usd = amount / rates[from_currency]
    final_amount = amount_in_usd * rates[to_currency]
    return f"{amount} {from_currency} is approximately {final_amount:.2f} {to_currency}"

currency_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="currency_converter",
            description="Converts a monetary amount from one currency to another",
            parameters=types.Schema(
                type="OBJECT",
                properties={
                    "amount": types.Schema(
                        type="NUMBER", 
                        description="The amount of money to convert"
                    ),
                    "from_currency": types.Schema(
                        type="STRING", 
                        description="The currency code to convert from (ex. USD)"
                    ),
                    "to_currency": types.Schema(
                        type="STRING", 
                        description="The currency code to convert to (ex. JPY)"
                    ),
                },
                required=["amount", "from_currency", "to_currency"]
            )
        )
    ]
)

console.print("tools configured: google search & currency converter")

tools configured: google search & currency converter

In [4]:
class Agent:
    def __init__(self, name, model, system_instruction, tools=None, tool_map=None):
        self.name = name
        self.model = model
        self.system_instruction = system_instruction
        self.tools = tools
        self.tool_map = tool_map or {}
        self.history = []

    def generate_response(self, prompt, context=None):
        full_prompt = f"PREVIOUS CONTEXT:\n{context}\n\nCURRENT TASK:\n{prompt}" if context else prompt
        self.history.append(types.Content(role="user", parts=[types.Part(text=full_prompt)]))
        console.print(f"{self.name} is at work.")

        try:
            config = types.GenerateContentConfig(
                system_instruction=self.system_instruction,
                tools=self.tools,
                temperature=0.7
            )
            response = client.models.generate_content(model=self.model, contents=self.history, config=config)
            part = response.candidates[0].content.parts[0]

            if part.function_call:
                fc = part.function_call
                fn_name = fc.name
                fn_args = fc.args
                console.print(f"calling tool: {fn_name}({fn_args})")
                tool_result = self.tool_map.get(fn_name, lambda **kwargs: f"Error: Tool '{fn_name}' not found.")(**fn_args)
                fn_response_part = types.Part.from_function_response(name=fn_name, response={"result": tool_result})
                self.history.append(types.Content(role="user", parts=[fn_response_part]))
                return self.generate_response(prompt, context)

            response_text = part.text
            self.history.append(response.candidates[0].content)
            return response_text

        except Exception as e:
            console.print(f"error in {self.name}: {e}")
            return f"error encountered: {e}"

In [5]:
# instantiate agents
MODEL_ID = "gemini-2.5-flash-lite" 

# research agent
researcher = Agent(
    name="Research Agent",
    model=MODEL_ID,
    system_instruction="""
    You are a Travel Researcher AI. Your goal is to gather accurate, up-to-date information about a destination. 
    1. Use Google Search to find real-time information. 
    2. Focus on collecting: 
       - Top 3 tourist attractions (with short descriptions). 
       - 1 hidden gem or off-the-beaten-path location. 
       - Recommended local foods or dishes. 
    3. Return the results in a structured, factual list. 
    4. Avoid personal opinions or filler text. 
    5. Keep the output concise but detailed enough for a Trip Planner to use directly.
    """,
    tools=[google_search_tool],
    tool_map={}
)

# tool map
tool_mapping = {
    "currency_converter": currency_converter,
    "get_rates": get_rates
}

# plan agent
planner = Agent(
    name="Plan Agent",
    model=MODEL_ID,
    system_instruction="""
    You are a Trip Planner AI. Your goal is to create a day-by-day travel itinerary based on research input. 
    1. ALWAYS use the 'currency_converter' tool for any budget or price calculations. 
       - Do NOT write conversions in plain text. 
       - The first step should always be a structured tool call to 'currency_converter'. 
       - Read the tool's response and then continue planning. 
    2. Create a day-by-day itinerary with clear headings for each day. 
    3. Include recommended attractions, meals, and activities while respecting the user's budget. 
    4. Keep the weather at the destination in mind while planning.
    5. Format the output entirely in Markdown for easy reading. 
    6. Avoid filler sentences, introductions, or repeated budget explanations. 
    7. Ensure all monetary amounts use the tool for accurate conversions.
    """,
    tools=[currency_tool],
    tool_map=tool_mapping
)

console.print("agents ready & linked")

agents ready & linked

In [6]:
# workflow & orchestration
def run_tripverse(destination, duration, interests, budget, budget_curr):
    console.print(f"TripVerse: Planning a {duration} trip to {destination}")
    
    research_task = f"find things to do in {destination} for {interests}. check weather for the duration of the trip."
    research_data = researcher.generate_response(research_task)
    
    planning_task = f"create a {duration} itinerary. the user has a budget of ${budget} {budget_curr}. convert this to the local currency. keep the budget in mind for the itinerary"
    
    final_plan = planner.generate_response(planning_task, context=research_data)
    
    console.print(f"Itinerary For {destination}")
    console.print(Markdown(final_plan))

In [7]:
# test file
destination = 'Iceland'
duration = "3 days"
interests = "Hiking, Exploration, and Sight Seeing"
budget = 5000
budget_curr = "usd"

run_tripverse(destination, duration, interests, budget, budget_curr)

TripVerse: Planning a 3 days trip to Iceland

Research Agent is at work.

2025-11-30 21:49:24,858 - google_genai.models - AFC is enabled with max remote calls: 10.
2025-11-30 21:49:29,001 - httpx - HTTP Request: POST https://dp.kaggle.net/palmapi/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


Plan Agent is at work.

2025-11-30 21:49:29,607 - httpx - HTTP Request: POST https://dp.kaggle.net/palmapi/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


calling tool: currency_converter({'amount': 5000, 'to_currency': 'ISK', 'from_currency': 'USD'})

Plan Agent is at work.

2025-11-30 21:49:32,343 - httpx - HTTP Request: POST https://dp.kaggle.net/palmapi/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"


Itinerary For Iceland

3-Day Iceland Winter Itinerary                                           

Budget: 639,726.00 ISK                                                                                             

                                             Day 1: The Golden Circle                                              

 • Morning: Explore Þingvellir National Park, a UNESCO World Heritage site. Walk between the tectonic plates and   
   learn about Iceland's history.                                                                                  
 • Lunch: Enjoy traditional Icelandic lamb soup (Kjötsúpa) at a local restaurant near the Geysir geothermal area.  
 • Afternoon: Witness the Strokkur geyser erupt at the Geysir geothermal area. Then, marvel at the power of        
   Gullfoss waterfall.                                                                                             
 • Evening: Return to Reykjavik. Dinner at a restaurant serving Plokkfiskur (fish stew).                           

                                            Day 2: South Coast Wonders                                             

 • Morning: Drive along the South Coast to Seljalandsfoss waterfall. Experience the unique opportunity to walk     
   behind the cascading water.                                                                                     
 • Lunch: Picnic lunch with Skyr and rye bread (consider purchasing these at a local grocery store to save money). 
 • Afternoon: Visit Skógafoss waterfall. For a less crowded experience, hike to Kvernufoss waterfall, located a    
   short distance away.                                                                                            
 • Evening: Enjoy dinner in a town along the South Coast, or return to Reykjavik.                                  

                                         Day 3: Glacier Lagoon & Departure                                         

 • Morning: Travel to Jökulsárlón Glacier Lagoon. Take a boat tour (weather permitting) among the icebergs. Keep an
   eye out for seals.                                                                                              
 • Lunch: Light lunch at a cafe near Jökulsárlón.                                                                  
 • Afternoon: Begin your journey back towards Reykjavik, with a stop at a black sand beach if time and weather     
   permit.                                                                                                         
 • Evening: Farewell dinner in Reykjavik.                                                                          

Note: Winter weather in Iceland can be unpredictable. Always check road conditions and weather forecasts before    
traveling. Flexibility is key for a winter itinerary. Consider booking accommodations and tours in advance,        
especially during peak season.